In [1]:
from tvae.synthesizers.tvae import TVAE

from sklearn.model_selection import train_test_split

import numpy as np

import pandas as pd
import torch

In [2]:
df = pd.read_csv("../../data/full_all_dataset_x.csv", sep=",")
binary_columns = [col for col in df.columns if df[col].dtype == "int64"]
df[binary_columns] = df[binary_columns].astype("category")

n_samples = df.shape[0]
n_features = df.shape[1]
fit_idx, val_idx = train_test_split(np.arange(0, n_samples, 1), shuffle=True)

fit = torch.from_numpy(df.iloc[fit_idx, :].values).to(torch.float)
val = torch.from_numpy(df.iloc[val_idx, :].values).to(torch.float)


In [3]:
inp_size = fit.size(-1)
compress_dims = tuple([inp_size//i for i in range(1, 4)]),

model = TVAE(
    embedding_dim=2,
    compress_dims=compress_dims,
    decompress_dims=compress_dims[::-1],
    l2scale=1e-5,
    batch_size=64,
    epochs=100,
    loss_factor=2,
    cuda=True
)



In [4]:
from sdv.metadata import SingleTableMetadata


metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)


In [5]:
model.fit(fit.cpu().numpy(), binary_columns)